# Let's define a preprocessing pipiline to use it with different configs to check impact

In [6]:
# imports
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from pymystem3 import Mystem
from difflib import SequenceMatcher as SM
from nltk.util import ngrams
import codecs

import demoji

from sklearn.feature_extraction.text import TfidfVectorizer

import logging
logging.basicConfig(
    level=logging.INFO,
    format=' %(asctime)s - %(message)s',
    datefmt='%H:%M')

printlog = True

def log(msg):
    if printlog:
        logging.info(msg)

import itertools
import string
from fuzzywuzzy import fuzz, process

# https://github.com/asyncee/python-obscene-words-filter
from obscene_words_filter import conf
from obscene_words_filter.words_filter import ObsceneWordsFilter

# from sqlalchemy import create_engine

hdir = './'
odir = f'{hdir}obtain/'
sdir = f'{hdir}scrub/'
mdir = f'{hdir}model/'

def preprocess_comments(new_comments, config):
    '''
    main function used in this notebook
    it reads raw comments, and makes preprocessing using instructions
    in config file, and returns preprocessed comments
    
    in: 
        comments // columns = [['review_id', 'content']]
        config
    out: 
        t[['review_id', 'text', 'content']] (text colunm added)
    '''
    
    print()
    print(config)
    print()

    # obscene filter based on regular expressions
    f = ObsceneWordsFilter(conf.bad_words_re, conf.good_words_re)

    s_all = ['п еиё з д',
    'х у йёяию',
    'о х у е втл',
    'п и д оеа р',
    'п и д р',
    'её б а нклт',
    'её б н у т',
    'у её б оа нтк',
    'её б л аои',
    'в ы её б',
    'е б еуё т',
    'св ъь еёи б',
    'б л я',
    'г оа в н',
    'м у д а кч',
    'г ао н д о н',
    'г н д о н,'
    'ч м оы',
    'д е р ь м',
    'ш л ю х',
    'з ао л у п',
    'м ао н д',
    'с у ч а р',
    'д ао л б ао её б']

    obs = []
    for i in s_all:
        s = list(itertools.product(*i.split()))
        for j in s:
            obs.append(''.join(j))

    def swear2(phrase0, obs):
        '''
        inexact search based obscene words filter
        '''
        if not config['obscene']: return 0
        phrase = phrase0.lower()
        for i in obs:
             if fuzz.partial_ratio(i, phrase) > 75:
                    return 1
        return 0

    cols = ['created', 'content', 'rating', 'review_id']
    c3 = new_comments[cols].copy()


    # lemmatization from yandex
    # https://stackoverflow.com/a/68047265/6950776
    mystem = Mystem()
    from string import punctuation
    punctuation0 = ',.;'
    punctuation = punctuation.replace('_', '').replace('-', '')
    for i in punctuation0:
        punctuation = punctuation.replace(i, '')

    russian_stopwords = stopwords.words("russian")
    russian_stopwords.pop(russian_stopwords.index('не'))

    # punctuation symbols to exclude, inclusing numbers
    s = '0123456789'
    for i in list(s):
        punctuation += i

    # stopwords to exclude
    russian_stopwords.append('чо')
    russian_stopwords.append('чтоле')
    russian_stopwords.append('штоле')
    russian_stopwords.append('вж')
    russian_stopwords.append('почему')
    russian_stopwords.append('бк')
    russian_stopwords.append('эх')
    russian_stopwords.append('что')
    russian_stopwords.append('пр')
    russian_stopwords.append('мой')
    russian_stopwords.append('самый')
    russian_stopwords.append('свой')
    russian_stopwords.append('этот')
    russian_stopwords.append('это')
    russian_stopwords.append('за')
    russian_stopwords.append('из')

    # misleading words when changing form. we use form we need forcingly
    repl_dict = {'еду':'еда',
                'тупить':'лагать',
                'плачу':'платить',
                'заработало':'работать',
                'бургер кинг':'бк',
                'bk':'бк',
                'burger king':'бк',
                'до сих пор':'досихпор',
                'так себе':'таксебе'
    }

    # business knowledge replacements
    coupons = pd.read_excel(f'{odir}coupons.xlsx')
    coupons = coupons['combo_name'].values.tolist()
    coupons = [i.lower() for i in coupons]

    def find_substring(needle, hay):
    #     https://stackoverflow.com/a/31433394/6950776
    
        '''
        this function searches for a group of words 
        using inexact search, and replaces it with predefined word synonim
        '''

        needle_length  = len(needle.split())
        max_sim_val    = 0
        max_sim_string = u""

        for ngram in ngrams(hay.split(), needle_length + int(.2*needle_length)):
            hay_ngram = u" ".join(ngram)
            similarity = SM(None, hay_ngram, needle).ratio()
            if similarity > max_sim_val:
                max_sim_val = similarity
                max_sim_string = hay_ngram

        return max_sim_val, max_sim_string

    def clean_comments(txt, len_thresh=2, verbose=False):
        
        '''
        subfunction for lemmatization, 
        uses all defined above and config
        '''

        if config['emoji']:
            candidate = demoji.replace(txt.lower(), "")
            
        if config['replace_business']:
            (coupon, simil) = process.extractOne(candidate, coupons, scorer=fuzz.token_set_ratio)
            if simil > 88:
                max_sim_val, max_sim_string = find_substring(coupon, candidate)
                candidate = candidate.replace(max_sim_string, 'купон')
        s = candidate
        
        if config['lemmatize']:
            if config['replace_business']:
                for i in repl_dict.keys():
                    s = s.replace(i, repl_dict[i])
            for i in punctuation:
                if i in s:
                    s = s.replace(i, ' ')
            for i in punctuation0:
                if i in s:
                    s = s.replace(i, ' ')
            if verbose: print(1,s)
            s = s.replace('  ', ' ')
            s = f.mask_bad_words(s).replace('*', '').split(' ')
            if verbose: print(1,s)
            s = [i for i in s if (i != '')and(i not in russian_stopwords) and (swear2(i, obs)==0)]
            if verbose: print(11,s)
            s = mystem.lemmatize(" ".join(s))
            if verbose: print(1,s)
            s = [i for i in s if not i == ' ']
            s = ' '.join(s)
            if verbose: print(2,s)

            s = s.replace(' - ', '_')
            s = s.replace('не ', 'не_').split(' ')
            if verbose: print(3,s)

            s = [i for i in s if (not len(i) < len_thresh)]
            if verbose: print(4,s)
            if len(s)<2: s = []
        else: s = s.split()
        return s

    log('clean and lemmatize comments')
    c3['cleaned_txt'] = c3['content'].apply(lambda x: clean_comments(x))

    t = c3.copy()
    t['comment'] = t['cleaned_txt']
    s = t['comment']

    corpus = []
    for i in s:
        corpus.append(' '.join(i))

    # attempt to leave only frequent words
    if config['tfidf_thresh'] > 0:
        log('compute TF-IDF and clean on threshold')
        # https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(corpus)
        feature_names = vectorizer.get_feature_names_out()
        dense = vectors.todense()
        denselist = dense.tolist()
        df = pd.DataFrame(denselist, columns=feature_names)

        # be careful with sum
        s = df.sum().reset_index().set_axis(['term', 'freq'], axis=1).sort_values('freq', ascending=False)

        thresh = config['tfidf_thresh']
        shorts = s.query('freq > @thresh').assign(l = lambda row: row['term'].str.len()).\
            sort_values('l').query('l < 4 and term != "смс"')['term'].values.tolist()
        terms = s.query('freq > @thresh')['term'].values.tolist()
        
    else:
        terms = set()
        for i in corpus:
            for j in i.split():
                terms.add(j)
        terms = list(terms)
        shorts = [i for i in terms if i != 'смс' or len(i) < 4]

    # data-driven stopwords
    sw = ['что','пр','мой','самый','свой','этот','это','за','из',
        'сразу', 'новый', 'ужас', 'ужасный', 'ужасно', 'пока',
        'хрен', 'пипец', 'конченый','дебильный','дико','днище','быдло',
        'дрянь','фаст','весь', 'нигде','кроме', 'очень', 'тупить', 'тупо',
        'king','ммммм','блин','бомж','безобразие','безобразно','бесить',
        'бесконечно', 'бесконечный', 'бред', 'беспредел', 'еще', 'даун', 'либо'
         ]

    terms = [i for i in terms if (i not in shorts)and(i not in sw)]


    '''
    s is a set of unique terms. if word in s, we use it.
    if word is in synonims of s, we also use it.
    let's construct dict named di with synonims of s.
    if a word in di, then it has parent word from s in its value
    '''

    log('find synonyms')
    # https://habr.com/ru/post/491448/
    s = terms.copy()
    di = {}
    while terms:
        candidate = terms.pop(0)

        resid = terms.copy() #residual
        while resid:
            r = resid.pop(0)
            if r == candidate: continue
            if fuzz.token_set_ratio(r, candidate) > 88:
                terms.pop(terms.index(r))

                if r in di.keys():
                    di[r] += candidate
                else:
                    di[r] = candidate

    def comment_clean(lst):
        res = []
        for i in lst:
            if not config['sinonyms']:
                res.append(i)
            if i in s:
                res.append(i)
            elif i in di.keys():
                res.append(di[i])
        return res

    log('leave meaningful words and their synonyms')
    t['comment_clean'] = t['comment'].apply(lambda x: comment_clean(x))

    def clean_short(txt):
        s = demoji.replace(txt.lower(), "")
        for i in repl_dict.keys():
            s = s.replace(i, repl_dict[i])
        for i in punctuation:
            if i in s:
                s = s.replace(i, '')
        for i in punctuation0:
            if i in s:
                s = s.replace(i, ' ')
        return s.split()

    t.drop(columns=['cleaned_txt', 'comment'], inplace=True)
    t.rename(columns={'comment_clean':'cleaned_txt'}, inplace=True)

    log('short preprocessing of short comments')
    t0 = t.copy()
    t0['l'] = t['cleaned_txt'].apply(lambda x: len(x))
    t1 = t0.loc[t0['l']>0].copy()
    t0 = t0.loc[t0['l']==0].copy()
    t0['cleaned_txt'] = t0['content'].apply(lambda x: clean_short(x))
    t = pd.concat([t0, t1], ignore_index=True)
    t['text'] = t['cleaned_txt'].apply(lambda x: ' '.join(x).replace('_', ' ').replace('досихпор', 'до сих пор').replace('таксебе', 'так себе'))

    return t[['review_id', 'text', 'content']]

# get texts, preprocessed with different configs (combinations of preprocessing rules)

In [7]:
%%time 

config1 = {
    'emoji':1,
    'obscene':0,
    'lemmatize':0,
    'replace_business':0,
    'sinonyms':0,
    'tfidf_thresh':0
}

config2 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':0,
    'replace_business':0,
    'sinonyms':0,
    'tfidf_thresh':0
}

config3_0 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':1,
    'replace_business':0,
    'sinonyms':0,
    'tfidf_thresh':0
}

config3 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':1,
    'replace_business':1,
    'sinonyms':0,
    'tfidf_thresh':0
}

config4 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':1,
    'replace_business':1,
    'sinonyms':0,
    'tfidf_thresh':0.7
}

config5 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':1,
    'replace_business':1,
    'sinonyms':1,
    'tfidf_thresh':0.7
}

config6 = {
    'emoji':1,
    'obscene':1,
    'lemmatize':1,
    'replace_business':0,
    'sinonyms':1,
    'tfidf_thresh':0.7
}


t = pd.read_excel(f'{sdir}t4m_2022_05_05__21_59.xlsx')
t['review_id'] = t.index

p1 = preprocess_comments(t, config1)
p2 = preprocess_comments(t, config2)
p3_0 = preprocess_comments(t, config3_0)
p3 = preprocess_comments(t, config3)
p4 = preprocess_comments(t, config4)
p5 = preprocess_comments(t, config5)
p6 = preprocess_comments(t, config6)


 20:49 - clean and lemmatize comments



{'emoji': 1, 'obscene': 0, 'lemmatize': 0, 'replace_business': 0, 'sinonyms': 0, 'tfidf_thresh': 0}



 20:49 - find synonyms
 20:49 - leave meaningful words and their synonyms
 20:49 - short preprocessing of short comments
 20:49 - clean and lemmatize comments



{'emoji': 1, 'obscene': 1, 'lemmatize': 0, 'replace_business': 0, 'sinonyms': 0, 'tfidf_thresh': 0}



 20:49 - find synonyms
 20:49 - leave meaningful words and their synonyms
 20:49 - short preprocessing of short comments
 20:49 - clean and lemmatize comments


...


{'emoji': 1, 'obscene': 1, 'lemmatize': 1, 'replace_business': 1, 'sinonyms': 1, 'tfidf_thresh': 0.7}



 20:50 - Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '…']
 20:50 - compute TF-IDF and clean on threshold
 20:51 - find synonyms
 20:51 - leave meaningful words and their synonyms
 20:51 - short preprocessing of short comments
 20:51 - clean and lemmatize comments



{'emoji': 1, 'obscene': 1, 'lemmatize': 1, 'replace_business': 0, 'sinonyms': 1, 'tfidf_thresh': 0.7}



 20:51 - compute TF-IDF and clean on threshold
 20:51 - find synonyms
 20:51 - leave meaningful words and their synonyms
 20:51 - short preprocessing of short comments


CPU times: user 2min 12s, sys: 3.9 s, total: 2min 16s
Wall time: 2min 27s


In [95]:
t

,created,content,rating,class_name,cleaned_txt,l,text,review_id
0,2021-10-31,Telegram,1,другое,['telegram'],0,telegram,0
1,2022-03-24,you cancelled the coupons,2,другое,"['you', 'cancelled', 'the', 'coupons']",0,you cancelled the coupons,1
2,2022-03-21,Does not load menu.,1,другое,"['does', 'not', 'load', 'menu']",0,does not load menu,2
3,2021-10-10,Не скачиваеться,1,другое,"['не', 'скачиваеться']",0,не скачиваеться,3
4,2022-03-18,Nie chcecie wycofac sie z rosji to wycofajcie ...,1,другое,"['nie', 'chcecie', 'wycofac', 'sie', 'z', 'ros...",0,nie chcecie wycofac sie z rosji to wycofajcie ...,4
...,...,...,...,...,...,...,...,...
3173,2022-04-10,"Покупка через онлайн не работает, короны не ис...",1,лояльность,"['покупка', 'онлайн', 'не_работать', 'корона',...",6,покупка онлайн не работать корона ошибка выдавать,3173
3174,2022-04-16,"Не реально заказать что-то за короны, хороший ...",3,лояльность,"['нереально', 'заказывать', 'корона', 'хороший']",4,нереально заказывать корона хороший,3174
3175,2021-09-28,Надоело при кааааждом заходе в приложение выби...,1,uxui,"['надоедать', 'заход', 'приложение', 'выбирать...",8,надоедать заход приложение выбирать снова снов...,3175
3176,2021-10-03,При каждом входе спрашивает ресторан для заказ...,1,uxui,"['каждый', 'вход', 'спрашивать', 'ресторан', '...",18,каждый вход спрашивать ресторан заказ раздража...,3176


In [8]:
cols = ['review_id', 'text']
res = t.merge(p1[cols], how='inner', on='review_id').\
        merge(p2[cols], how='inner', on='review_id').\
        merge(p3_0[cols], how='inner', on='review_id').\
        merge(p3[cols], how='inner', on='review_id').\
        merge(p5[cols], how='inner', on='review_id').\
        merge(p6[cols], how='inner', on='review_id').\
    set_axis(['created', 'content', 'rating', 'class_name', 'cleaned_txt', 'l',
       'text', 'review_id', 'text_1', 'text_2', 'text_3_0', 'text_3', 'text_5', 'text_6'], axis=1)
res

/var/folders/_r/mpnhfz9d4nv5tm_1d27p_36w0000gn/T/ipykernel_28404/2590779159.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'text_x'} in the result is deprecated and will raise a MergeError in a future version.
  res = t.merge(p1[cols], how='inner', on='review_id').\
/var/folders/_r/mpnhfz9d4nv5tm_1d27p_36w0000gn/T/ipykernel_28404/2590779159.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'text_x'} in the result is deprecated and will raise a MergeError in a future version.
  res = t.merge(p1[cols], how='inner', on='review_id').\


,created,content,rating,class_name,cleaned_txt,l,text,review_id,text_1,text_2,text_3_0,text_3,text_5,text_6
0,2021-10-31,Telegram,1,другое,['telegram'],0,telegram,0,telegram,telegram,telegram,telegram,telegram,telegram
1,2022-03-24,you cancelled the coupons,2,другое,"['you', 'cancelled', 'the', 'coupons']",0,you cancelled the coupons,1,you cancelled the coupons,you cancelled the coupons,you cancelled the coupons,you cancelled the coupons,you cancelled the coupons,you cancelled the coupons
2,2022-03-21,Does not load menu.,1,другое,"['does', 'not', 'load', 'menu']",0,does not load menu,2,does not load menu.,does not load menu.,does not load menu,does not load menu,does not load menu,does not load menu
3,2021-10-10,Не скачиваеться,1,другое,"['не', 'скачиваеться']",0,не скачиваеться,3,не скачиваеться,не скачиваеться,не скачиваеться,не скачиваеться,не скачиваеться,не скачиваеться
4,2022-03-18,Nie chcecie wycofac sie z rosji to wycofajcie ...,1,другое,"['nie', 'chcecie', 'wycofac', 'sie', 'z', 'ros...",0,nie chcecie wycofac sie z rosji to wycofajcie ...,4,nie chcecie wycofac sie z rosji to wycofajcie ...,nie chcecie wycofac sie z rosji to wycofajcie ...,nie chcecie wycofac sie rosji to wycofajcie si...,nie chcecie wycofac sie rosji to wycofajcie si...,nie chcecie wycofac sie z rosji to wycofajcie ...,nie chcecie wycofac sie z rosji to wycofajcie ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,2022-04-10,"Покупка через онлайн не работает, короны не ис...",1,лояльность,"['покупка', 'онлайн', 'не_работать', 'корона',...",6,покупка онлайн не работать корона ошибка выдавать,3173,"покупка через онлайн не работает, короны не ис...","покупка через онлайн не работает, короны не ис...",покупка онлайн не работать корона не использов...,покупка онлайн не работать корона не использов...,покупка онлайн не работать корона ошибка выдавать,покупка онлайн не работать корона ошибка выдавать
3174,2022-04-16,"Не реально заказать что-то за короны, хороший ...",3,лояльность,"['нереально', 'заказывать', 'корона', 'хороший']",4,нереально заказывать корона хороший,3174,"не реально заказать что-то за короны, хороший ...","не реально заказать что-то за короны, хороший ...",не реально заказывать что-то корона хороший ход,не реально заказывать что-то корона хороший ход,не реально заказывать корона хороший,не реально заказывать корона хороший
3175,2021-09-28,Надоело при кааааждом заходе в приложение выби...,1,uxui,"['надоедать', 'заход', 'приложение', 'выбирать...",8,надоедать заход приложение выбирать снова снов...,3175,надоело при кааааждом заходе в приложение выби...,надоело при кааааждом заходе в приложение выби...,надоедать кааааждый заход приложение выбирать ...,надоедать кааааждый заход приложение выбирать ...,надоедать заход приложение выбирать снова снов...,надоедать заход приложение выбирать снова снов...
3176,2021-10-03,При каждом входе спрашивает ресторан для заказ...,1,uxui,"['каждый', 'вход', 'спрашивать', 'ресторан', '...",18,каждый вход спрашивать ресторан заказ раздража...,3176,при каждом входе спрашивает ресторан для заказ...,при каждом входе спрашивает ресторан для заказ...,каждый вход спрашивать ресторан заказ раздража...,каждый вход спрашивать ресторан заказ раздража...,каждый вход спрашивать ресторан заказ раздража...,каждый вход спрашивать ресторан заказ раздража...


In [9]:
res.to_excel(f'{sdir}t4m_2022_05_05__21_59_v3.xlsx', index=False) # save to file to use in nb2 and further